# Dataset shift

In the previous assignment you have worked with wafer map data from a semiconductor manufacturing process, and developed a calibrated binary classifier for identifying whether a finished wafer contains a known pattern of faulty chips.

Imagine the company has evaluated several classifiers and deployed one of them in production, as part of a larger system for identifying the root causes of faulty chips. The root cause analysis relies on the expertise of human operators. When the root cause of faulty chips on a wafer can be identified, the process parameters can be adjusted, reducing the number of faulty chips and increasing wafer yield.

Ideally, the company would like all wafers to be analyzed by the operators, however, their capacity is limited. Therefore, the classifier is used as part of a filtering stage to select wafers which are suitable for further analysis by the operators, based on whether the wafers contain a known pattern of faulty chips.

After a month, the company observes degrading wafer yield. They suspect it is due to a drop in classification performance of the classifier: missing wafers that are suitable for further analysis will prevent the operators from identifying the root causes of faulty chips and adapting the process parameters, resulting in more faults and lower yield. Your task is to investigate this suspicion.

#### Overview

You are given two datasets: a labeled *training* set that represents the data used to train the classifier and a partially labeled *production* set representing the data collected after the model was deployed in production. Both datasets are based on the WM-811K dataset containing annotated wafer maps collected from real-world fabrication (see `./data/readme.txt`).

You are also given the implementation of the deployed classifier.

The assignment consists of two parts:

- Part 1 is about defining the data mining objective.
- Part 2 is about addressing the data mining objective.

#### Deliverables

There are two deliverables for this assignment:

- Report: it should be max 4 pages long (double column). It's structure should follow the CRISP-DM process. A template is provided in the assignment on Canvas. The report will be graded.
- This notebook: it must contain all the supporting material for the report, i.e. all the results and figures must be generated in this notebook. This notebook will not be graded, however, missing supporting material for the report will affect the grade of the report.

Deliver a `.zip` archive with three files:

- `dataset_shift_<groupid>.pdf` file containing the report,
- `dataset_shift_<groupid>.ipynb` file containing this filled in **and executed** notebook,
- `dataset_shift_<groupid>.html` file containing this filled in **and executed** notebook rendered to html,

by submitting it to the corresponding assignment on Canvas. `<groupid>` refers to your group number on Canvas. You may submit as many times as you like before the deadline. The last submission counts.

Throughout this notebook you will find cells starting `# // BEGIN_TODO`.

- Fill in all your code between these `# // BEGIN_TODO [Q0]` and `# // END_TODO [Q0]` tags.
- Do not edit these tags in any way, or else your answers may not be processed by the grading system.
- You can add as many code and text cells as you want between the `# // BEGIN_TODO [Q0]` and `# // END_TODO [Q0]` tags to make your code nicely readable.
- Unlike the previous assignment, your submitted notebook will *not* be evaluated by Momotor. You are free to use any libraries you like, but make sure you import all your libraries between the `# // BEGIN_TODO [Q0]` and `# // END_TODO [Q0]` tags.

You are encouraged to play with the data and extend this notebook in order to obtain your answers. You may insert cells at any point in the notebook, but remember:
<br/><br/>
<div style="padding: 15px; border: 1px solid transparent; border-color: transparent; margin-bottom: 20px; border-radius: 4px; color: #a94442; background-color: #f2dede; border-color: #ebccd1;
">
Only the code in your answer cells (i.e. between `# // BEGIN_TODO` and `# // END_TODO`) will be extracted and evaluated.
</div>

<div style="padding: 15px; border: 1px solid transparent; border-color: transparent; margin-bottom: 20px; border-radius: 4px; color: #a94442; background-color: #f2dede; border-color: #ebccd1;
">
Before delivering your notebook, make sure that the cells in your notebook can be executed in sequence without errors, by executing "Restart & Run All" from the "Kernel" menu.
</div>

Let's get started by filling in your details in the following answer cell. Assign your group number, your names and student ids to variables `group_number`, `name_student1`, `id_student1`, `name_student2`, `id_student2`, e.g.:

```
# // BEGIN_TODO [AUTHOR]
group_number = "7"
name_student1 = "John Smith"
id_student1 = "1234567"
name_student2 = "Jane Miller"
id_student2 = "7654321"
# // END_TODO [AUTHOR]
```

In [ ]:
#// BEGIN_TODO [AUTHOR]

# ===== =====> Replace this line by your code. <===== ===== #

#// END_TODO [AUTHOR]

## Import libraries

You are free to use any libraries you like. However, please import any additional libraries between `# // BEGIN_TODO [CODE]` and `# // END_TODO [CODE]` tags in Part 2.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

## Load the data

The training data, that was used to train the classifier before deployment, resides in the `./data/wafer_training.pkl` pickle file.

In [ ]:
with open('./data/wafer_training.pkl', 'rb') as f:
    X_train, y_train = pickle.load(f)

`X_train` contains the wafer maps and `y_train` contains the labels. The maps are 26x26 pixels, a pixel of value 0 represents the background, a pixel of value 1 indicates a good chip, and a pixel of value 2 indicates a bad chip. An example of a wafer map is shown below.

In [ ]:
fig, axes = plt.subplots(1, figsize = (2,2))
plt.imshow(X_train[0]);

The production data, that was collected after the classifier was deployed in production, resides in the `./data/wafer_production.pkl` pickle file.

In [ ]:
with open('./data/wafer_production.pkl', 'rb') as f:
    X_production, y_production = pickle.load(f)

`X_production` contains the wafer maps and `y_production` contains the labels. The samples are listed in the same order as they were collected.

The wafer maps were labeled by human experts according to whether the wafer map contains a known bad chip pattern or not:

- 1: a pattern (one of the 8 patterns in the Calibration assignment)
- 0: no pattern
- -1: unlabeled sample

You will observe that `y_production` is only partially labeled, i.e. some samples are unlabeled. You can imagine that labeling is time consuming and expensive process and only a subset was labeled, e.g. for monitoring the performance of the classifier.

An example of the wafer map with and without known pattern is shown below.

In [ ]:
unique_classes, class_indexes = np.unique(y_train,return_index=True)
class_names = ["No pattern", "Pattern"]
fig, axes = plt.subplots(1,2, figsize = (6,2))
for num_index, index in enumerate(class_indexes):
    axes[num_index].imshow(X_train[index])
    axes[num_index].set_title(class_names[unique_classes[num_index]])
    axes[num_index].set_xticks([])
    axes[num_index].set_yticks([])

## Define the classifier

Here is the implementation of the deployed classifier. It is a multi-layer perceptron with one hidden layer containing 10 neurons, that is trained using the Adam optimizer with early stopping. It expects a vector as input.

In [ ]:
from sklearn.neural_network import MLPClassifier

deployed_classifier = MLPClassifier(hidden_layer_sizes=(10), early_stopping=True, tol=1e-4, validation_fraction=0.1,
                                n_iter_no_change=20, solver="adam", max_iter=100, random_state=1)

## Part 1: identify the data mining objective

Think of a data mining objective that can be derived from the description above. The objective should be general enough to prevent similar problems from occurring in the future. We will discuss your objectives during the lab session on October 6th and decide on a common objective to be addressed by all groups (it will be posted on Teams for those unable to attend the lab session). Feel free to explore the data.

In [ ]:
# TODO: Explore the data

## Part 2: address the data mining objective

Address the *common* data mining objective and document your findings in the report. You are free to use any approach you like. To get started you may read the required reading in the slides on Dataset Shift and follow the cited papers.

<br/>
<div style="padding: 15px; border: 1px solid transparent; border-color: transparent; margin-bottom: 20px; border-radius: 4px; color: #a94442; background-color: #f2dede; border-color: #ebccd1;
">
For this assignment only your report will be graded. While your code will not be graded directly, any results and figures in your report must be generated by your code below.
</div>

In [ ]:
#// BEGIN_TODO [CODE] (0 points)

# ===== =====> Replace this line by your code. <===== ===== #


In [ ]:
#// END_TODO [CODE]

# Feedback

Please fill in this questionaire to help us improve this course for the next year. Your feedback will be anonymized and will not affect your grade in any way!

### How many hours did you spend on this assignment?

Assign a number to variable `feedback_time`.

In [ ]:
#// BEGIN_FEEDBACK [Feedback_1] (0 points)

#// END_FEEDBACK [Feedback_1]

import numbers
assert isinstance(feedback_time, numbers.Number), "Please assign a number to variable feedback_time"

### How difficult did you find this assignment?

Assign an integer to variable `feedback_difficulty`, on a scale 0 - 10, with 0 being very easy, 5 being just right, and 10 being very difficult.

In [ ]:
#// BEGIN_FEEDBACK [Feedback_2] (0 points)

#// END_FEEDBACK [Feedback_2]

assert isinstance(feedback_difficulty, numbers.Number), "Please assign a number to variable feedback_difficulty"

### (Optional) What did you like?

Assign a string to variable `feedback_like`.

In [ ]:
#// BEGIN_FEEDBACK [Feedback_5] (0 points)

#// END_FEEDBACK [Feedback_5]

### (Optional) What can be improved?

Assign a string to variable `feedback_improve`. Please be specific, so that we can act on your feedback.

In [ ]:
#// BEGIN_FEEDBACK [Feedback_6] (0 points)

#// END_FEEDBACK [Feedback_6]